# Setup

In [1]:
# import packages
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import sys
import matplotlib
from tensorflow import keras
import datetime
import matplotlib.pyplot as plt

In [2]:
# import support libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, \
                                                    img_to_array, load_img
                                                    
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, \
                                        BatchNormalization, GlobalAveragePooling2D

from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16, Xception

In [3]:
print('Version check:')
print('Python: {}'.format(sys.version))
print('pandas: {}'.format(pd.__version__))
print('NumPy: {}'.format(np.__version__))
print('sklearn: {}'.format(sklearn.__version__))
print('matplotlib: {}'.format(matplotlib.__version__))
print('TensorFlow: {}'.format(tf.__version__))
print('Keras: {}'.format(keras.__version__))

Version check:
Python: 3.8.2 (default, Apr 27 2020, 15:53:34) 
[GCC 9.3.0]
pandas: 1.0.3
NumPy: 1.18.3
sklearn: 0.22.2.post1
matplotlib: 3.2.1
TensorFlow: 2.2.0
Keras: 2.3.0-tf


In [4]:
# Backend Settings

# clear Keras session
keras.backend.clear_session()

# set seeds
np.random.seed(18)
tf.random.set_seed(18)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))
#tf.debugging.set_log_device_placement(True)

Num GPUs Available:  1


# Global Functions

In [5]:
class TrainRuntimeCallback(keras.callbacks.Callback):

  def on_train_begin(self,logs={}):
    self.start = datetime.datetime.now()

  def on_train_end(self,logs={}):
    self.process_time = (datetime.datetime.now() - self.start).total_seconds()

class TestRuntimeCallback(keras.callbacks.Callback):

  def on_test_begin(self,logs={}):
    self.start = datetime.datetime.now()

  def on_test_end(self,logs={}):
    self.process_time = (datetime.datetime.now() - self.start).total_seconds()

In [6]:
def train_model(model, optimizer, train_input, val_input, len_df, model_name):
    data = dict()

    # Compile model
    model.compile(optimizer = optimizer,
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    # Create a callback to record training time
    train_rt = TrainRuntimeCallback()

    # Model fitting parameters
    history = model.fit(
        train_input,
        steps_per_epoch = len_df // batch_size,
        #steps_per_epoch = 32,
        epochs = 20,
        callbacks = [train_rt],
        validation_data=val_input,
        validation_steps = (len_df // batch_size)
        #validation_steps = 32
    )

    train_time = train_rt.process_time
    #print(train_time)

    history_dict = history.history

    data['model'] = model_name
    data['train_time'] = train_time
    data['train_loss'] = history_dict['loss'][-1]
    data['train_acc'] = history_dict['accuracy'][-1]
    data['val_loss'] = history_dict['val_loss'][-1]
    data['val_acc'] = history_dict['val_accuracy'][-1]

    return data, history_dict

In [7]:
def test_model(model, test_input, len_df):
    
    data = dict()

    # Create test callback
    test_rt = TestRuntimeCallback()

    test_loss, test_acc = model.evaluate(
        test_input,
        steps = len_df // batch_size,
        callbacks = [test_rt]
    )
    test_time = test_rt.process_time
    data['test_time'] = test_time
    data['test_loss'] = test_loss
    data['test_acc'] = test_acc

    return data

In [8]:
def round_val(val):
    return round(val, 3)

In [9]:
def save_model_data(train_data, test_data):
    data = dict()

    data['model'] = train_data['model']
    data['train_loss'] = round_val(train_data['train_loss'])
    data['train_acc'] = round_val(train_data['train_acc'])
    data['train_time'] = round_val(train_data['train_time'])
    data['val_loss'] = round_val(train_data['val_loss'])
    data['val_acc'] = round_val(train_data['val_acc'])
    data['test_loss'] = round_val(test_data['test_loss'])
    data['test_acc'] = round_val(test_data['test_acc'])
    data['test_time'] = round_val(test_data['test_time'])

    return data

# Load and import data

In [10]:
train_dir = 'data_files/train/'
test_dir = 'data_files/test/'

train_df = pd.read_csv('data_files/train.csv')
test_df = pd.read_csv('data_files/test.csv')

train_df = train_df.sort_values('filename')
test_df = test_df.sort_values('filename')

In [11]:
train_df.head()

,Unnamed: 0,id_code,experiment,plate,well,sirna,filename
0,0,HEPG2-01_1_B03,HEPG2-01,1,B03,513,HEPG2-01_1_B03_s1.jpeg
36515,36515,HEPG2-01_1_B03,HEPG2-01,1,B03,513,HEPG2-01_1_B03_s2.jpeg
1,1,HEPG2-01_1_B04,HEPG2-01,1,B04,840,HEPG2-01_1_B04_s1.jpeg
36516,36516,HEPG2-01_1_B04,HEPG2-01,1,B04,840,HEPG2-01_1_B04_s2.jpeg
2,2,HEPG2-01_1_B05,HEPG2-01,1,B05,1020,HEPG2-01_1_B05_s1.jpeg


In [12]:
test_df.head()

,Unnamed: 0,well_id,experiment,plate,well,filename,sirna_id
0,0,HEPG2-08_1_B03,HEPG2-08,1,B03,HEPG2-08_1_B03_s1.jpeg,855
19897,39794,HEPG2-08_1_B03,HEPG2-08,1,B03,HEPG2-08_1_B03_s2.jpeg,855
1,2,HEPG2-08_1_B04,HEPG2-08,1,B04,HEPG2-08_1_B04_s1.jpeg,710
19898,39796,HEPG2-08_1_B04,HEPG2-08,1,B04,HEPG2-08_1_B04_s2.jpeg,710
2,4,HEPG2-08_1_B05,HEPG2-08,1,B05,HEPG2-08_1_B05_s1.jpeg,836


# Preprocessing

## Select one cell line at a time

In [13]:
df = pd.read_csv('rxrx1/rxrx1.csv')
df.head()

,site_id,well_id,cell_type,dataset,experiment,plate,well,site,well_type,sirna,sirna_id
0,HEPG2-08_1_B02_1,HEPG2-08_1_B02,HEPG2,test,HEPG2-08,1,B02,1,negative_control,EMPTY,1138
1,HEPG2-08_1_B02_2,HEPG2-08_1_B02,HEPG2,test,HEPG2-08,1,B02,2,negative_control,EMPTY,1138
2,HEPG2-08_1_B03_1,HEPG2-08_1_B03,HEPG2,test,HEPG2-08,1,B03,1,treatment,s21721,855
3,HEPG2-08_1_B03_2,HEPG2-08_1_B03,HEPG2,test,HEPG2-08,1,B03,2,treatment,s21721,855
4,HEPG2-08_1_B04_1,HEPG2-08_1_B04,HEPG2,test,HEPG2-08,1,B04,1,treatment,s20894,710


In [14]:
cell_types = list(df.cell_type.unique())
cell_types

['HEPG2', 'HUVEC', 'RPE', 'U2OS']

In [15]:
train_df.columns

Index(['Unnamed: 0', 'id_code', 'experiment', 'plate', 'well', 'sirna',
       'filename'],
      dtype='object')

In [16]:
hpeg_train_df = train_df[train_df.experiment.str.contains(cell_types[0])]
huvec_train_df = train_df[train_df.experiment.str.contains(cell_types[1])]
rpe_train_df = train_df[train_df.experiment.str.contains(cell_types[2])]
u2os_train_df = train_df[train_df.experiment.str.contains(cell_types[3])]

hpeg_test_df = test_df[test_df.experiment.str.contains(cell_types[0])]
huvec_test_df = test_df[test_df.experiment.str.contains(cell_types[1])]
rpe_test_df = test_df[test_df.experiment.str.contains(cell_types[2])]
u2os_test_df = test_df[test_df.experiment.str.contains(cell_types[3])]

In [17]:
len(hpeg_test_df), len(hpeg_test_df)

(8858, 8858)

In [18]:
len(huvec_train_df), len(huvec_test_df)

(35376, 17692)

In [19]:
len(rpe_train_df), len(rpe_test_df)

(15506, 8834)

In [20]:
len(u2os_train_df), len(u2os_test_df)

(6648, 4410)

In [21]:
len(hpeg_test_df.sirna_id.unique())

1108

## Create validation dfs for each cell type

In [22]:
hpeg_val_count = int(len(hpeg_train_df) * 0.5)
huvec_val_count = int(len(huvec_train_df) * 0.5)
rpe_val_count = int(len(rpe_train_df) * 0.5)
u2os_val_count = int(len(u2os_train_df) * 0.5)

hpeg_val_count, huvec_val_count, rpe_val_count, u2os_val_count

(7750, 17688, 7753, 3324)

In [23]:
def create_val_set(train_df, val_count):
    cell_val_df = train_df.sample(val_count, random_state = 18)
    cell_file_list = list(cell_val_df.filename)
    cell_val_df = train_df[train_df.filename.isin(cell_file_list)]
    cell_train_df = train_df[~train_df.filename.isin(cell_file_list)]

    return cell_train_df, cell_val_df

In [24]:
# HPEG2
hpeg_val_df = hpeg_train_df.sample(hpeg_val_count, random_state = 18)
hpeg_file_list = list(hpeg_val_df.filename)
hpeg_val_df = hpeg_train_df[hpeg_train_df.filename.isin(hpeg_file_list)]
hpeg_val_list = hpeg_train_df[~hpeg_train_df.filename.isin(hpeg_file_list)]

len(hpeg_train_df), len(hpeg_val_df)


(15500, 7750)

In [25]:
huvec_train_df, huvec_val_df = create_val_set(huvec_train_df, huvec_val_count)
rpe_train_df, rpe_val_df = create_val_set(rpe_train_df, rpe_val_count)
u2os_train_df, u2os_val_df = create_val_set(u2os_train_df, u2os_val_count)

In [26]:
len(huvec_train_df), len(huvec_val_df)

(17688, 17688)

In [27]:
len(rpe_train_df), len(rpe_val_df)

(7753, 7753)

In [28]:
len(u2os_train_df), len(u2os_val_df)

(3324, 3324)

## Check if each train and validation set has all labels

In [29]:
len(hpeg_train_df.sirna.unique()), len(hpeg_val_df.sirna.unique())

(1108, 1108)

In [30]:
len(huvec_train_df.sirna.unique()), len(huvec_val_df.sirna.unique())

(1108, 1108)

In [31]:
len(rpe_train_df.sirna.unique()), len(rpe_val_df.sirna.unique())

(1108, 1108)

In [32]:
len(u2os_train_df.sirna.unique()), len(u2os_val_df.sirna.unique()) # skip this cell line

(1088, 1090)

## Change labels to strings

In [33]:
# HPEG2
hpeg_train_df.sirna = hpeg_train_df.sirna.apply(lambda x: str(x))
hpeg_val_df.sirna = hpeg_val_df.sirna.apply(lambda x: str(x))
hpeg_test_df.sirna_id = hpeg_test_df.sirna_id.apply(lambda x: str(x))

# huvec
huvec_train_df.sirna = huvec_train_df.sirna.apply(lambda x: str(x))
huvec_val_df.sirna = huvec_val_df.sirna.apply(lambda x: str(x))
huvec_test_df.sirna_id = huvec_test_df.sirna_id.apply(lambda x: str(x))

# RPE
rpe_train_df.sirna = rpe_train_df.sirna.apply(lambda x: str(x))
rpe_val_df.sirna = rpe_val_df.sirna.apply(lambda x: str(x))
rpe_test_df.sirna_id = rpe_test_df.sirna_id.apply(lambda x: str(x))

# U2OS
u2os_train_df.sirna = u2os_train_df.sirna.apply(lambda x: str(x))
u2os_val_df.sirna = u2os_val_df.sirna.apply(lambda x: str(x))
u2os_test_df.sirna_id = u2os_test_df.sirna_id.apply(lambda x: str(x))

## Settings

In [34]:
# Settings
batch_size = 32
img_height = 224
img_width = 224
num_outputs = 1108
epochs = 20

# Train and test models

In [35]:
results = list()

## HPEG2

### Image Augmentation Settings

In [36]:
# Add some rotation and adjustments to images

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    #validation_split = 0.2, # set validation set to 0.2
    #featurewise_center= True,
    #featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    #height_shift_range=[-0.08, 0.08],
    width_shift_range=[-0.08,0.08],
    #brightness_range=[0.75, 1.1],
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator  = train_datagen.flow_from_dataframe(
    dataframe = hpeg_train_df,
    directory = train_dir,
    target_size = (img_height, img_width),
    subset='training',
    x_col='filename',
    y_col='sirna',
    class_mode='categorical',
    color_mode='rgb',
    shuffle = True,
    batch_size = batch_size
)

val_generator = test_datagen.flow_from_dataframe(
    dataframe = hpeg_val_df,
    directory = train_dir,
    target_size = (img_height, img_width),
    #subset = 'validation',
    x_col = 'filename',
    y_col = 'sirna',
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = False,
    batch_size = batch_size
)

test_dir = '/home/specc/Documents/school_files/458_deep_learning/458_final_project/data_files/test/'

test_generator = test_datagen.flow_from_dataframe(
    dataframe = hpeg_test_df,
    directory = test_dir,
    target_size = (224, 224),
    x_col='filename',
    y_col='sirna_id',
    mode='categorical',
    color_mode='rgb'
)

Found 15500 validated image filenames belonging to 1108 classes.
Found 7750 validated image filenames belonging to 1108 classes.
Found 8858 validated image filenames belonging to 1108 classes.


### InceptionV3 with GlobalAveragePooling

In [37]:
num_outputs = len(hpeg_train_df.sirna.unique())
print(num_outputs)

1108


In [38]:
len(hpeg_val_df.sirna.unique())

1108

In [39]:
base_model = InceptionV3(include_top=False,
                    weights = 'imagenet',
                    input_shape=(img_height, img_width, 3))

pooling = GlobalAveragePooling2D()
flat = Flatten()
output = Dense(num_outputs, activation='softmax')

model = Sequential([
    base_model,
    pooling,
    flat,
    output
])

model_name = 'HPEG - InceptionV3 w GlobalAvgPooling & width shift'

for layer in base_model.layers:
    layer.trainable = False

In [40]:
opt = RMSprop(learning_rate = 0.0005, momentum = 0.9)

train_data, history_dict = train_model(model, opt, train_generator, 
                                            val_generator, len(hpeg_train_df), model_name)

Epoch 1/20
484/484 [==============================] - 652s 1s/step - loss: 10.8935 - accuracy: 0.0037 - val_loss: 9.8256 - val_accuracy: 0.0113
Epoch 2/20
484/484 [==============================] - 650s 1s/step - loss: 10.8587 - accuracy: 0.0113 - val_loss: 9.4760 - val_accuracy: 0.0209
Epoch 3/20
484/484 [==============================] - 651s 1s/step - loss: 10.8421 - accuracy: 0.0173 - val_loss: 9.5076 - val_accuracy: 0.0266
Epoch 4/20
484/484 [==============================] - 653s 1s/step - loss: 10.8471 - accuracy: 0.0215 - val_loss: 9.3151 - val_accuracy: 0.0358
Epoch 5/20
484/484 [==============================] - 654s 1s/step - loss: 10.8529 - accuracy: 0.0245 - val_loss: 9.4011 - val_accuracy: 0.0367
Epoch 6/20
484/484 [==============================] - 653s 1s/step - loss: 10.8176 - accuracy: 0.0299 - val_loss: 9.4087 - val_accuracy: 0.0456
Epoch 7/20
484/484 [==============================] - 652s 1s/step - loss: 10.8492 - accuracy: 0.0326 - val_loss: 9.4121 - val_accuracy:

In [41]:
test_data = test_model(model, test_generator, len(hpeg_train_df))
results.append(save_model_data(train_data, test_data))

484/484 [==============================] - 317s 656ms/step - loss: 15.2280 - accuracy: 0.0162


## HUVEC

In [42]:
# Add some rotation and adjustments to images

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    #validation_split = 0.2, # set validation set to 0.2
    #featurewise_center= True,
    #featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    #height_shift_range=[-0.08, 0.08],
    width_shift_range=[-0.08,0.08],
    #brightness_range=[0.75, 1.1]
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator  = train_datagen.flow_from_dataframe(
    dataframe = huvec_train_df,
    directory = train_dir,
    target_size = (img_height, img_width),
    subset='training',
    x_col='filename',
    y_col='sirna',
    class_mode='categorical',
    color_mode='rgb',
    shuffle = True,
    batch_size = batch_size
)

val_generator = test_datagen.flow_from_dataframe(
    dataframe = huvec_val_df,
    directory = train_dir,
    target_size = (img_height, img_width),
    #subset = 'validation',
    x_col = 'filename',
    y_col = 'sirna',
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = False,
    batch_size = batch_size
)

test_dir = '/home/specc/Documents/school_files/458_deep_learning/458_final_project/data_files/test/'

test_generator = test_datagen.flow_from_dataframe(
    dataframe = huvec_test_df,
    directory = test_dir,
    target_size = (224, 224),
    x_col='filename',
    y_col='sirna_id',
    mode='categorical',
    color_mode='rgb'
)

Found 17688 validated image filenames belonging to 1108 classes.
Found 17688 validated image filenames belonging to 1108 classes.
Found 17692 validated image filenames belonging to 1108 classes.


In [43]:
base_model = InceptionV3(include_top=False,
                    weights = 'imagenet',
                    input_shape=(img_height, img_width, 3))

pooling = GlobalAveragePooling2D()
flat = Flatten()
output = Dense(num_outputs, activation='softmax')

model = Sequential([
    base_model,
    pooling,
    flat,
    output
])

model_name = 'HUVEC - InceptionV3 w GlobalAvgPooling & width shift'

for layer in base_model.layers:
    layer.trainable = False

In [44]:
opt = RMSprop(learning_rate = 0.0005, momentum = 0.9)

train_data, history_dict = train_model(model, opt, train_generator, 
                                            val_generator, len(huvec_train_df), model_name)

Epoch 1/20
552/552 [==============================] - 726s 1s/step - loss: 9.9740 - accuracy: 0.0087 - val_loss: 10.7341 - val_accuracy: 0.0125
Epoch 2/20
552/552 [==============================] - 721s 1s/step - loss: 9.9078 - accuracy: 0.0218 - val_loss: 11.0243 - val_accuracy: 0.0145
Epoch 3/20
552/552 [==============================] - 722s 1s/step - loss: 10.0599 - accuracy: 0.0288 - val_loss: 11.3417 - val_accuracy: 0.0187
Epoch 4/20
552/552 [==============================] - 721s 1s/step - loss: 10.0917 - accuracy: 0.0359 - val_loss: 11.5358 - val_accuracy: 0.0211
Epoch 5/20
552/552 [==============================] - 721s 1s/step - loss: 10.1780 - accuracy: 0.0399 - val_loss: 11.3201 - val_accuracy: 0.0267
Epoch 6/20
552/552 [==============================] - 722s 1s/step - loss: 10.1826 - accuracy: 0.0434 - val_loss: 12.0448 - val_accuracy: 0.0222
Epoch 7/20
552/552 [==============================] - 723s 1s/step - loss: 10.2422 - accuracy: 0.0450 - val_loss: 11.9675 - val_accu

In [45]:
test_data = test_model(model, test_generator, len(huvec_test_df))
results.append(save_model_data(train_data, test_data))

552/552 [==============================] - 358s 649ms/step - loss: 13.8875 - accuracy: 0.0236


In [47]:
pd.DataFrame(results)

,model,train_loss,train_acc,train_time,val_loss,val_acc,test_loss,test_acc,test_time
0,HPEG - InceptionV3 w GlobalAvgPooling & width ...,11.072,0.057,13051.280,9.958,0.068,15.228,0.016,318.123
1,HUVEC - InceptionV3 w GlobalAvgPooling & width...,10.576,0.074,14464.989,13.873,0.027,13.888,0.024,359.096
